In [1]:
import numpy as np
from sklearn.utils import check_random_state
from sklearn.model_selection import train_test_split

import matplotlib.pyplot as plt

%matplotlib qt

fontsize = 13
params = {'axes.labelsize': fontsize + 2,
      'font.size': fontsize + 2,
      'legend.fontsize': fontsize,
      'xtick.labelsize': fontsize,
      'ytick.labelsize': fontsize,
      'text.usetex': True}
plt.rcParams.update(params)

In [2]:
def get_data(n_samples=100, n_features=2, sigma=1e-1, sigma_X=1e-2,
             random_state=1):

    rng = check_random_state(random_state)
    u = rng.randn(n_samples, 1)
    u /= np.linalg.norm(u)
    X = u + sigma_X * rng.randn(n_samples, n_features)

    theta = rng.randn(n_features)
    y = X@theta + sigma * rng.randn(n_samples)

    X_train, X_test, y_train, y_test = train_test_split(
        X, y, random_state=rng
    )

    return X_train, y_train, X_test, y_test, rng

In [3]:
def soba(X_train, y_train, X_test, y_test, z0=None, v0=None, x0=None, 
         max_iter=100_000, step_size_in=1., outer_ratio=.01, 
         adaptative_step_size=1, alpha=1):
    n_train, n_features = X_train.shape
    n_test = X_test.shape[0]
    L = np.linalg.norm(X_train, ord=2)**2 / X_train.shape[0]
    H = X_train.T.dot(X_train)/n_train

    z = z0
    v = v0
    x = x0
    step_size_out = step_size_in / outer_ratio

    z_list, v_list, x_list = [], [], []
    for k in range(max_iter):
        grad_in_z = X_train.T.dot(X_train @ z - y_train) / n_train + np.exp(x) * z
        Hess = H + np.exp(x) * np.eye(n_features)
        hvp = Hess @ v
        cross_v = np.exp(x) * z * v

        grad_out_z = X_test.T.dot(X_test @ z - y_test) / n_test
        if adaptative_step_size == 1:
            step_size_in = alpha / (L + np.exp(x).max())
        elif adaptative_step_size == 2:
            step_size_in = alpha / (L + np.exp(x))
        z -= step_size_in * grad_in_z
        v -= step_size_in * (hvp - grad_out_z)
        x -= step_size_out * (- cross_v)

        z_list.append(z.copy())
        v_list.append(v.copy())
        x_list.append(x.copy())
    return np.array(z_list), np.array(v_list), np.array(x_list)

In [4]:
def amigo(X_train, y_train, X_test, y_test, z0=None, v0=None, x0=None,
          max_iter=100, n_inner_step=10, n_v_step=10,
          step_size_in=1, outer_ratio=.01):
    
    n_train, n_features = X_train.shape
    n_test = X_test.shape[0]

    z = z0
    v = v0
    x = x0
    step_size_out = step_size_in / outer_ratio
    H = X_train.T.dot(X_train)/n_train

    z_list, v_list, x_list = [], [], []
    for _ in range(max_iter):

        for _ in range(n_inner_step):
            grad_in_z = X_train.T.dot(X_train @ z - y_train) / n_train + np.exp(x) * z
            z -= step_size_in * grad_in_z

        # compute SGD for the auxillary variable
        for _ in range(n_v_step):
            Hess = H + np.exp(x) * np.eye(n_features)
            hvp = Hess @ v
            grad_out_z = X_test.T.dot(X_test @ z - y_test) / n_test
            v -= step_size_in * (hvp - grad_out_z)


        cross_v = np.exp(x) * z * v
        x -= step_size_out * (- cross_v)

        z_list.append(z.copy())
        v_list.append(v.copy())
        x_list.append(x.copy())

    return np.array(z_list), np.array(v_list), np.array(x_list)

In [5]:
n_samples, n_features = 100, 2
max_iter = 100_000

step_size_in = 1
outer_ratio = .01

X_train, y_train, X_test, y_test, rng = get_data(n_samples=n_samples, n_features=n_features, random_state=1)
H = X_train.T.dot(X_train)/X_train.shape[0]

rng = check_random_state(2442)
z_list, v_list, x_list = soba(X_train, y_train, X_test, y_test, z0=rng.randn(n_features),
                              v0=np.zeros(n_features), x0=np.log(rng.rand(n_features)), max_iter=max_iter,
                              step_size_in=step_size_in, outer_ratio=outer_ratio, adaptative_step_size=False)

z_star_list = np.array([np.linalg.solve(H + np.diag(np.exp(x)), X_train.T.dot(y_train) / X_train.shape[0]) for x in x_list])
v_star_list = np.array([np.linalg.solve(H + np.diag(np.exp(x)), X_test.T.dot(X_test@z - y_test) / X_test.shape[0]) for z, x in zip(z_list, x_list)])
value_function = np.array([np.linalg.norm(X_test @ z - y_test)**2/X_test.shape[0] for z in z_star_list])

<ipython-input-3-009f6bda3c97>:16: RuntimeWarning: overflow encountered in exp
  grad_in_z = X_train.T.dot(X_train @ z - y_train) / n_train + np.exp(x) * z
<ipython-input-3-009f6bda3c97>:17: RuntimeWarning: overflow encountered in exp
  Hess = H + np.exp(x) * np.eye(n_features)
<ipython-input-3-009f6bda3c97>:17: RuntimeWarning: invalid value encountered in multiply
  Hess = H + np.exp(x) * np.eye(n_features)
<ipython-input-3-009f6bda3c97>:19: RuntimeWarning: overflow encountered in exp
  cross_v = np.exp(x) * z * v
<ipython-input-3-009f6bda3c97>:26: RuntimeWarning: invalid value encountered in subtract
  z -= step_size_in * grad_in_z
<ipython-input-5-c356e3e15e30>:15: RuntimeWarning: overflow encountered in exp
  z_star_list = np.array([np.linalg.solve(H + np.diag(np.exp(x)), X_train.T.dot(y_train) / X_train.shape[0]) for x in x_list])


In [6]:
rng = check_random_state(2442)
n_inner_step = 10
n_v_step = 10
z_list_amigo, v_list_amigo, x_list_amigo = amigo(X_train, y_train, X_test, y_test, z0=rng.randn(n_features),
                              v0=np.zeros(n_features), x0=np.log(rng.rand(n_features)), max_iter=max_iter,
                              step_size_in=step_size_in, outer_ratio=outer_ratio, n_inner_step=n_inner_step,
                              n_v_step=n_v_step)

z_star_list_amigo = np.array([np.linalg.solve(H + np.diag(np.exp(x)), X_train.T.dot(y_train) / X_train.shape[0]) for x in x_list_amigo])
v_star_list_amigo = np.array([np.linalg.solve(H + np.diag(np.exp(x)), X_test.T.dot(X_test@z - y_test) / X_test.shape[0]) for z, x in zip(z_list_amigo, x_list_amigo)])
value_function_amigo = np.array([np.linalg.norm(X_test @ z - y_test)**2/X_test.shape[0] for z in z_star_list_amigo])

In [7]:
rng = check_random_state(2442)
z_list_a_max, v_list_a_max, x_list_a_max = soba(X_train, y_train, X_test, y_test, z0=rng.randn(n_features),
                              v0=np.zeros(n_features), x0=np.log(rng.rand(n_features)), max_iter=max_iter,
                              step_size_in=step_size_in, outer_ratio=outer_ratio, adaptative_step_size=2)

z_star_list_a_max = np.array([np.linalg.solve(H + np.diag(np.exp(x)), X_train.T.dot(y_train) / X_train.shape[0]) for x in x_list_a_max])
v_star_list_a_max = np.array([np.linalg.solve(H + np.diag(np.exp(x)), X_test.T.dot(X_test@z - y_test) / X_test.shape[0]) for z, x in zip(z_list_a_max, x_list_a_max)])
value_function_a_max = np.array([np.linalg.norm(X_test @ z - y_test)**2/X_test.shape[0] for z in z_star_list_a_max])

In [8]:
rng = check_random_state(2442)
z_list_a, v_list_a, x_list_a = soba(X_train, y_train, X_test, y_test, z0=rng.randn(n_features),
                              v0=np.zeros(n_features), x0=np.log(rng.rand(n_features)), max_iter=max_iter,
                              step_size_in=step_size_in, outer_ratio=outer_ratio, adaptative_step_size=1)

z_star_list_a = np.array([np.linalg.solve(H + np.diag(np.exp(x)), X_train.T.dot(y_train) / X_train.shape[0]) for x in x_list_a])
v_star_list_a = np.array([np.linalg.solve(H + np.diag(np.exp(x)), X_test.T.dot(X_test@z - y_test) / X_test.shape[0]) for z, x in zip(z_list_a, x_list_a)])
value_function_a = np.array([np.linalg.norm(X_test @ z - y_test)**2/X_test.shape[0] for z in z_star_list_a])

In [9]:
def h(x, X_train, y_train, X_test, y_test):
    n_train, n_features = X_train.shape
    n_test = X_test.shape[0]
    z_star  = np.linalg.solve(X_train.T.dot(X_train)/n_train + np.diag(np.exp(x)), X_train.T.dot(y_train) / n_train)
    return .5 * np.linalg.norm(X_test @ z_star - y_test) / n_test

In [10]:
plt.figure()
# plt.semilogy(np.linalg.norm(z_list - z_star_list, axis=1), label="SOBA, constant step size")
plt.semilogy(np.linalg.norm(z_list_a_max - z_star_list_a_max, axis=1), label=r"SOBA, step size : $\frac{1}{L+\|e^x\|_\infty}$")
plt.semilogy(np.linalg.norm(z_list_a - z_star_list_a, axis=1), label=r"SOBA, step size : $\frac{1}{L+e^x}$ ")
plt.semilogy(np.linalg.norm(z_list_amigo - z_star_list_amigo, axis=1), label=r"Amigo, constant step size")
plt.xlabel("Iterations")
plt.ylabel(r"$\|z^t-z^*(x^t)\|$")
plt.legend()
plt.tight_layout()
plt.show()

In [11]:
plt.figure()
# plt.semilogy(np.linalg.norm(v_list - v_star_list, axis=1), label="SOBA, constant step size")
plt.semilogy(np.linalg.norm(v_list_a - v_star_list_a, axis=1), label=r"SOBA, step size : $\frac{1}{L+\|e^x\|_\infty}$")
plt.semilogy(np.linalg.norm(v_list_a_max - v_star_list_a_max, axis=1), label=r"SOBA, step size : $\frac{1}{L+e^x}$")
plt.semilogy(np.linalg.norm(v_list_amigo - v_star_list_amigo, axis=1), label=r"Amigo, constant step size")
plt.xlabel("Iterations")
plt.ylabel(r"$\|v^t-v^*(x^t)\|$")
plt.legend()
plt.tight_layout()
plt.show()

In [12]:
plt.figure()
plt.semilogy(value_function, label="SOBA, constant step size", linewidth=2)
plt.semilogy(value_function_a, label=r"SOBA, step size : $\frac{1}{L+\|e^x\|_\infty}$", linewidth=2)
plt.semilogy(value_function_a_max, label=r"SOBA, step size : $\frac{1}{L+e^x}$", linewidth=2)
plt.semilogy(value_function_amigo, label=r"Amigo, constant step size", linewidth=2)
plt.xlabel("Iterations")
plt.ylabel("Value function")
plt.legend()
plt.tight_layout()
plt.show()

In [13]:
plt.figure()
plt.plot(x_list[:52000, 0], x_list[:52000, 1], label="SOBA, constant step size", linewidth=2)
plt.plot(x_list_a[:, 0], x_list_a[:, 1], label=r"SOBA, step size : $\frac{1}{L+\|e^x\|_\infty}$", linewidth=2)
plt.plot(x_list_a_max[:, 0], x_list_a_max[:, 1], label=r"SOBA, step size : $\frac{1}{L+e^x}$", linewidth=2)
plt.plot(x_list_amigo[:, 0], x_list_amigo[:, 1], label=r"Amigo, constant step size", linewidth=2)

plt.xlabel(r"$x_0$")
plt.ylabel(r"$x_1$")
plt.legend()
plt.tight_layout()
plt.show()

In [14]:
alphas = [.25, .5, .9, 1]
max_iter = 100_000
z_list = np.zeros((len(alphas), max_iter, 2))
v_list = np.zeros((len(alphas), max_iter, 2))
x_list = np.zeros((len(alphas), max_iter, 2))
z_star_list = np.zeros((len(alphas), max_iter, 2))
v_star_list = np.zeros((len(alphas), max_iter, 2))
value_function = np.zeros((len(alphas), max_iter))

z_list_max = np.zeros((len(alphas), max_iter, 2))
v_list_max = np.zeros((len(alphas), max_iter, 2))
x_list_max = np.zeros((len(alphas), max_iter, 2))
z_star_list_max = np.zeros((len(alphas), max_iter, 2))
v_star_list_max = np.zeros((len(alphas), max_iter, 2))
value_function_max = np.zeros((len(alphas), max_iter))

for i, alpha in enumerate(alphas):
    z_list[i], v_list[i], x_list[i] = soba(X_train, y_train, X_test, y_test, z0=rng.randn(n_features),
                                v0=np.zeros(n_features), x0=np.log(rng.rand(n_features)), max_iter=max_iter,
                                step_size_in=step_size_in, outer_ratio=outer_ratio, adaptative_step_size=2)
    z_star_list[i] = np.array([np.linalg.solve(H + np.diag(np.exp(x)), X_train.T.dot(y_train) / X_train.shape[0]) for x in x_list[i]])
    v_star_list[i] = np.array([np.linalg.solve(H + np.diag(np.exp(x)), X_test.T.dot(X_test@z - y_test) / X_test.shape[0]) for z, x in zip(z_list[i], x_list[i])])
    value_function[i] = np.array([np.linalg.norm(X_test @ z - y_test)**2/X_test.shape[0] for z in z_star_list[i]])

    z_list_max[i], v_list_max[i], x_list_max[i] = soba(X_train, y_train, X_test, y_test, z0=rng.randn(n_features),
                            v0=np.zeros(n_features), x0=np.log(rng.rand(n_features)), max_iter=max_iter,
                            step_size_in=step_size_in, outer_ratio=outer_ratio, adaptative_step_size=1)

    z_star_list_max[i] = np.array([np.linalg.solve(H + np.diag(np.exp(x)), X_train.T.dot(y_train) / X_train.shape[0]) for x in x_list_max[i]])
    v_star_list_max[i] = np.array([np.linalg.solve(H + np.diag(np.exp(x)), X_test.T.dot(X_test@z - y_test) / X_test.shape[0]) for z, x in zip(z_list_max[i], x_list_max[i])])
    value_function_max[i] = np.array([np.linalg.norm(X_test @ z - y_test)**2/X_test.shape[0] for z in z_star_list_max[i]])

In [15]:
m = min(value_function.min(), value_function_max.min())
# m = 0
plt.figure()
for i,alpha in enumerate(alphas):    
    plt.loglog(value_function[i] - m, '-', label=rf"SOBA, step size : $\frac{{{alpha}}}{{L+e^x}}$", linewidth=2)
    # plt.loglog(value_function_max[i] - m, '--', label=rf"SOBA, step size : $\frac{{{alpha}}}{{L+\|e^x\|_\infty}}$", linewidth=2)
plt.xlabel("Iterations")
plt.ylabel(rf"$h(x^t) - h^*$")
plt.legend()
plt.tight_layout()
plt.show()

In [16]:
plt.figure()
for i,alpha in enumerate(alphas):    
    plt.semilogy(np.linalg.norm(z_list[i] - z_star_list[i], axis=1) , '-', label=rf"SOBA, step size : $\frac{{{alpha}}}{{L+e^x}}$", linewidth=2)
    # plt.semilogy(np.linalg.norm(z_list_max[i] - z_star_list_max[i], axis=1), '--', label=rf"SOBA, step size : $\frac{{{alpha}}}{{L+\|e^x\|_\infty}}$", linewidth=2)
plt.xlabel("Iterations")
plt.ylabel(r"$\|z^t-z^*(x^t)\|$")
plt.legend()
plt.tight_layout()
plt.show()